# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.
Please follow the [documentation](https://www.freqtrade.io/en/stable/data-download/) for more details.

## Setup

### Change Working directory to repository root

In [22]:
import os
from pathlib import Path

# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = ""
i=0
try:
    os.chdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

/freqtrade


### Configure Freqtrade environment

In [23]:
from freqtrade.configuration import Configuration

# Customize these according to your needs.

# Initialize empty configuration object
# config = Configuration.from_files([])
# Optionally (recommended), use existing configuration file
config = Configuration.from_files(["user_data/config.json"])

# Define some constants
config["timeframe"] = "15m"
# Name of the strategy class
config["strategy"] = "MeanReversionStrategy"
# Location of the data
data_location = config["datadir"]
# Pair to analyze - Only use one pair here
pair = "SUI/USDT:USDT"
print(data_location)

/freqtrade/user_data/data/binance


In [24]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "feather",  # Make sure to update this to your data
                            candle_type=CandleType.FUTURES,
                            )

# Confirm success
print(f"Loaded {len(candles)} rows of data for {pair} from {data_location}")
candles.head()

Loaded 36191 rows of data for SUI/USDT:USDT from /freqtrade/user_data/data/binance


,date,open,high,low,close,volume
0,2023-05-03 16:00:00+00:00,1.4030,1.4030,1.2652,1.3338,11474534.0
1,2023-05-03 16:15:00+00:00,1.3339,1.3863,1.3327,1.3716,16090260.4
2,2023-05-03 16:30:00+00:00,1.3717,1.3957,1.3547,1.3857,13192974.2
3,2023-05-03 16:45:00+00:00,1.3866,1.3924,1.3652,1.3733,11951258.0
4,2023-05-03 17:00:00+00:00,1.3731,1.3756,1.3405,1.3574,12698235.4


## Load and run strategy
* Rerun each time the strategy file is changed

In [25]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
from freqtrade.data.dataprovider import DataProvider
strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {'pair': pair})
df.tail()

,date,open,high,low,close,volume,return,volume_threshold_16_49,wbb_upperband_36_2,wbb_lowerband_36_2,wbb_middleband_36_2,enter_tag,enter_long,enter_short,exit_tag
36186,2024-05-14 14:30:00+00:00,0.9190,0.9236,0.9088,0.9137,4596078.6,-0.005551,7.915209e+06,0.010172,-0.012721,-0.001274,,NaN,NaN,
36187,2024-05-14 14:45:00+00:00,0.9136,0.9136,0.9070,0.9098,2432348.7,-0.004268,7.994324e+06,0.009872,-0.012745,-0.001436,,NaN,NaN,
36188,2024-05-14 15:00:00+00:00,0.9099,0.9104,0.9024,0.9091,3116459.8,-0.000769,8.060373e+06,0.007808,-0.010608,-0.001400,,NaN,NaN,
36189,2024-05-14 15:15:00+00:00,0.9093,0.9174,0.9088,0.9142,2986349.9,0.005610,8.180112e+06,0.008318,-0.010361,-0.001021,,NaN,NaN,
36190,2024-05-14 15:30:00+00:00,0.9141,0.9141,0.9059,0.9065,1774524.4,-0.008423,8.208383e+06,0.007534,-0.010376,-0.001421,,NaN,NaN,


### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [15]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
data = df.set_index('date', drop=False)
data.tail()

Generated 357.0 entry signals


,date,open,high,low,close,volume,return,volume_threshold_16_49,wbb_upperband_36_2,wbb_lowerband_36_2,wbb_middleband_36_2,enter_tag,enter_long,enter_short,exit_tag
date,,,,,,,,,,,,,,,
2024-05-14 14:30:00+00:00,2024-05-14 14:30:00+00:00,0.9190,0.9236,0.9088,0.9137,4596078.6,-0.005551,7.915209e+06,0.010172,-0.012721,-0.001274,,NaN,NaN,
2024-05-14 14:45:00+00:00,2024-05-14 14:45:00+00:00,0.9136,0.9136,0.9070,0.9098,2432348.7,-0.004268,7.994324e+06,0.009872,-0.012745,-0.001436,,NaN,NaN,
2024-05-14 15:00:00+00:00,2024-05-14 15:00:00+00:00,0.9099,0.9104,0.9024,0.9091,3116459.8,-0.000769,8.060373e+06,0.007808,-0.010608,-0.001400,,NaN,NaN,
2024-05-14 15:15:00+00:00,2024-05-14 15:15:00+00:00,0.9093,0.9174,0.9088,0.9142,2986349.9,0.005610,8.180112e+06,0.008318,-0.010361,-0.001021,,NaN,NaN,
2024-05-14 15:30:00+00:00,2024-05-14 15:30:00+00:00,0.9141,0.9141,0.9059,0.9065,1774524.4,-0.008423,8.208383e+06,0.007534,-0.010376,-0.001421,,NaN,NaN,


## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [16]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2024-05-08_13-19-59.json"

In [17]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'MeanReversionStrategy'
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


[{'key': '1000PEPE/USDT:USDT', 'trades': 501, 'profit_mean': 0.005768021715644424, 'profit_mean_pct': 0.58, 'profit_sum': 2.8897788795378565, 'profit_sum_pct': 288.98, 'profit_total_abs': 5670.06515716, 'profit_total': 0.567006515716, 'profit_total_pct': 56.7, 'duration_avg': '4:55:00', 'wins': 372, 'draws': 1, 'losses': 128, 'winrate': 0.7425149700598802}, {'key': 'SUI/USDT:USDT', 'trades': 423, 'profit_mean': 0.006015941231581851, 'profit_mean_pct': 0.6, 'profit_sum': 2.544743140959123, 'profit_sum_pct': 254.47, 'profit_total_abs': 4144.1721468999995, 'profit_total': 0.4144172146899999, 'profit_total_pct': 41.44, 'duration_avg': '6:23:00', 'wins': 323, 'draws': 3, 'losses': 97, 'winrate': 0.7635933806146572}, {'key': 'LTC/USDT:USDT', 'trades': 490, 'profit_mean': 0.0016494723706392525, 'profit_mean_pct': 0.16, 'profit_sum': 0.8082414616132336, 'profit_sum_pct': 80.82, 'profit_total_abs': 1267.5999574999996, 'profit_total': 0.12675999574999997, 'profit_total_pct': 12.68, 'duration_avg

In [26]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

pair                exit_reason       
1000PEPE/USDT:USDT  roi                   411
                    stop_loss              89
                    trailing_stop_loss      1
LTC/USDT:USDT       roi                   439
                    stop_loss              50
                    trailing_stop_loss      1
SUI/USDT:USDT       roi                   363
                    stop_loss              60
Name: count, dtype: int64

## Plotting daily profit / equity line

In [27]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_stats
import plotly.express as px
import pandas as pd

strategy = 'MeanReversionStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats['strategy'][strategy]

df = pd.DataFrame(columns=['dates','equity'], data=strategy_stats['daily_profit'])
df['equity_daily'] = df['equity'].cumsum()

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["exit_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a separate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.